In [1]:
import sagemaker
import boto3
import io
import json
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
def read_s3_file(client, bucket_name, key):
    response = s3.get_object(Bucket=bucket_name, Key=key)
    data = response['Body'].read()
    return json.loads(data)

In [3]:
def process_file_data(data, columns):
    return pd.json_normalize(data['results'])[columns]

In [5]:
session = sagemaker.Session()

In [6]:
s3 = boto3.client('s3')

In [7]:
bucket = session.default_bucket()

In [8]:
key = 'data/raw/label/drug-label-0001-of-0010.json'

In [9]:
data = read_s3_file(s3, bucket, key)

In [4]:
columns = [
    'set_id',
    'id',
    'effective_time',
    'openfda.route',
    'openfda.substance_name',
    'openfda.product_type',
    'openfda.pharm_class_epc',
    'openfda.pharm_class_cs',
    'openfda.pharm_class_moa',
    'openfda.pharm_class_pe',
    'openfda.application_number'
]

In [12]:
df = process_file_data(data, columns=columns)

In [13]:
df.head()

,set_id,id,effective_time,openfda.route,openfda.substance_name,openfda.product_type,openfda.pharm_class_epc,openfda.pharm_class_cs,openfda.pharm_class_moa,openfda.pharm_class_pe,openfda.application_number
0,d1697161-ebe8-42b0-83e7-c9de4412546c,4dcb8674-48f7-4229-993d-bca9795b1676,20210824,[ORAL],[GALANTAMINE HYDROBROMIDE],[HUMAN PRESCRIPTION DRUG],NaN,NaN,NaN,NaN,[ANDA090178]
1,a582d57f-d191-4ca5-b105-585b2d8a0e3e,7519ffff-75f4-4ede-9ee9-b75870506b80,20190204,[ORAL],"[AMPHETAMINE ASPARTATE MONOHYDRATE, AMPHETAMIN...",[HUMAN PRESCRIPTION DRUG],NaN,NaN,NaN,NaN,[ANDA202424]
2,6401da8a-8cd5-72f6-e053-2991aa0ad5e0,7aa7efea-fa82-5a1f-e053-2a91aa0a5d7d,20181114,[TOPICAL],"[OCTINOXATE, TITANIUM DIOXIDE, ZINC OXIDE]",[HUMAN OTC DRUG],NaN,NaN,NaN,NaN,[part352]
3,ad529830-ad3f-4973-95cf-1e3ed5d15d26,ad529830-ad3f-4973-95cf-1e3ed5d15d26,20220308,[ORAL],[PHENOBARBITAL],[HUMAN PRESCRIPTION DRUG],NaN,NaN,NaN,NaN,NaN
4,6eb6ddac-fa26-43b3-bf3e-5b7b58138fdc,c52f0d41-044f-4e59-8c06-7c799c48baf7,20200731,[RESPIRATORY (INHALATION)],"[FLUTICASONE PROPIONATE, SALMETEROL XINAFOATE]",[HUMAN PRESCRIPTION DRUG],NaN,NaN,NaN,NaN,[NDA021077]
